### 0. Загрузка библиотек и моделей

In [1]:
!pip install deeppavlov
!python -m deeppavlov install ner_ontonotes_bert_mult_torch
!python -m deeppavlov install ner_rus
!pip install --upgrade pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pydantic-1.3-cp37-cp37m-manylinux2010_x86_64.whl (7.3 MB)
  Using cached pymorphy2-0.8-py2.py3-none-any.whl (46 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.2
    Uninstalling pydantic-1.9.2:
      Successfully uninstalled pydantic-1.9.2
  Attempting uninstall: pymorphy2
    Found existing installation: pymorphy2 0.9.1
    Uninstalling pymorphy2-0.9.1:
      Successfully uninstalled pymorphy2-0.9.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.1.0 requires pydantic!=1.8,!=1.8.1,<1.10.0,>=1.7.4, but you have pydantic 1.3 which is incompatible.
spacy 3.4.1 requires pydantic!=1.8,!=1.8.1,<1.10.0,>=1.7.4, but you have pydantic 1.3 which is incompatible.
ru-core-news-md 3.4.0 requires pymorphy2>=0.9, but yo

In [118]:
!pip install pipreqs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!python -m spacy download ru_core_news_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41.9 MB 1.3 MB/s 
  Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
  Using cached pydantic-1.9.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.1 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.3
    Uninstalling pydantic-1.3:
      Successfully uninstalled pydantic-1.3
  Attempting uninstall: pymorphy2
    Found existing installation: pymorphy2 0.8
    Uninstalling pymorphy2-0.8:
      Successfully uninstalled pymorphy2-0.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deeppavlov 0.17.4 requires pandas==0.25.3, but you have pandas 1.3.5 which is incompatible.
deeppavlov 0.17.4 requires pydantic==1.3, but you have pydantic 1.9.2 which is incompatible.
deeppavlov 0.17.4

In [3]:
import pandas as pd
from deeppavlov import build_model, configs
import spacy
import ru_core_news_md as model

In [5]:
import os
if not 'test_data.csv' in os.listdir():
  from google.colab import files

  uploaded = files.upload()

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

Saving test_data.csv to test_data.csv
User uploaded file "test_data.csv" with length 53427 bytes


### 1. Содержательная часть

Рассмотрим данные:

In [322]:
scripts = pd.read_csv('test_data.csv')
scripts.head()

,Unnamed: 0,dlg_id,line_n,role,text,insight
0,0,0,0,client,алло,"greeting=False, farewell=False, name=None, org=None"
1,1,0,1,manager,алло здравствуйте,"greeting=True, farewell=False, name=None, org=None"
2,2,0,2,client,добрый день,"greeting=False, farewell=False, name=None, org=None"
3,3,0,3,manager,меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,"greeting=False, farewell=False, name=ангелина, org=диджитал бизнес"
4,4,0,4,client,ага,"greeting=False, farewell=False, name=None, org=None"


In [321]:
scripts.loc[:, 'text'] = scripts.text.apply(str.lower)
scripts.head()

,dlg_id,line_n,role,text,insight
0,0,0,client,алло,"greeting=False, farewell=False, name=None, org=None"
1,0,1,manager,алло здравствуйте,"greeting=True, farewell=False, name=None, org=None"
2,0,2,client,добрый день,"greeting=False, farewell=False, name=None, org=None"
3,0,3,manager,меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,"greeting=False, farewell=False, name=ангелина, org=диджитал бизнес"
4,0,4,client,ага,"greeting=False, farewell=False, name=None, org=None"


Разные диалоги различаются по переменной dlg_id, реплики для проверки отмечены как 'manager' в столбце role.
Извлечем нумерацию из dlg_id, чтобы позже было удобно группировать фразы под иалогам:

In [123]:
dialogs_id = pd.unique(scripts.dlg_id)

Учитывая формальный характер беседы, а также то, что мы имеем дело со скриптом диалогов, для извлечения именованных сущностей первая идея - проверить реплики, содержащие фразы "меня зовут" (для имени):

In [69]:
lines_with_names = scripts[(scripts.text.str.contains('меня')) & (scripts.text.str.contains('зовут'))]
lines_with_names.text.values

array(['меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается',
       'меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается',
       'меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там',
       'добрый меня максим зовут компания китобизнес удобно говорить'],
      dtype=object)

В полученных репликах действительно есть имена, а также названия фирм. Заметно, что у этих реплик похожий паттерн - можно это использовать для извлечения именованных сущностей позже.

Возьмем эти реплики в качестве тестовых для трех моделей извлечения сущностей: мультиязыковой bert, Bi-LSTM модель для русского языка (deeppavlov) и spacy для русского языка.

In [77]:
ner_model_spacy = model.load()
doc = ner_model_spacy('\n'.join(scripts.text.values).lower())

ner_model_pavlov = build_model(configs.ner.ner_ontonotes_bert_mult_torch , download=True)
ner_model_p_rus = build_model(configs.ner.ner_rus , download=True)

2022-09-03 17:03:24.398 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/ner/ner_ontonotes_bert_mult_torch.tar.gz download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/v1/ner/ner_ontonotes_bert_mult_torch.tar.gz download because of matching hashes
2022-09-03 17:03:26.471 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_ontonotes_bert_mult_torch/bert-base-multilingual-cased/tag.dict]
INFO:deeppavlov.core.data.simple_vocab:[loading vocabulary from /root/.deeppavlov/models/ner_ontonotes_bert_mult_torch/bert-base-multilingual-cased/tag.dict]
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predicti

В данных не сохранился синтаксис и исходная капитализация слов, что могло бы облегчить обнаружение именованных сущностей. Чтобы как-то это компенсировать, протестируем модели на трех вариантах каждой реплики: состоящей полностью из маленьких букв, из больших букв, и из слов, где первая буква - заглавная.

In [117]:
all_lower = lines_with_names.text
all_upper = lines_with_names.text.apply(str.upper)
uppercase = lines_with_names.text.apply(lambda s: ' '.join([word.upper()[0] + word.lower()[1:] for word in s.split()]))

order_data = ['lower', 'upper', 'uppercase']
order_model = ['pavlov_multiligual', 'pavlov_rus', 'spacy_rus']

performance = pd.DataFrame(index = order_data, columns = order_model)
pd.set_option('display.max_colwidth', None)

for no, case in enumerate((all_lower, all_upper, uppercase)):

  for index, model in enumerate((ner_model_pavlov, ner_model_p_rus, ner_model_spacy)):

    if index == 2:

      entities = model('\n'.join(case.values)).ents
      performance.loc[order_data[no], order_model[index]] = [(e, e.label_) for e in entities]

    else:

      sents, labels = model(case.values)
      performance.loc[order_data[no], order_model[index]] = [ent for sent, lbls in (zip(sents,labels)) for ent in zip(sent, lbls) if ent[1] != 'O']
      
performance

,pavlov_multiligual,pavlov_rus,spacy_rus
lower,[],[],"[((ангелина), PER), ((ангелина), PER), ((ангелина), PER), ((максим), PER)]"
upper,[],"[(КОМПАНИЯ, B-ORG), (ДИДЖИТАЛ, I-ORG), (БИЗНЕС, I-ORG), (ЗВОНИМ, I-ORG), (ВАМ, B-ORG), (ВАС, B-ORG), (КОМПАНИЯ, B-ORG), (ДИДЖИТАЛ, I-ORG), (БИЗНЕС, I-ORG), (ВАМ, B-ORG), (ОБРАТИЛА, B-ORG), (ВНИМАНИЕ, I-ORG), (ЧТО, B-ORG), (ВАС, B-ORG), (ЗАКАНЧИВАЕТСЯ, B-LOC), (КОМПАНИЯ, B-ORG), (ДИДЖИТАЛ, I-ORG), (БИЗНЕС, I-ORG), (ВАМ, B-ORG), (МАКСИМ, B-ORG), (КОМПАНИЯ, B-ORG), (КИТОБИЗНЕС, I-ORG), (УДОБНО, I-ORG), (ГОВОРИТЬ, I-ORG)]","[((МЕНЯ, ЗОВУТ), ORG), ((ДИДЖИТАЛ, БИЗНЕС, ЗВОНИМ, ВАМ, ПО, ПОВОДУ, ПРОДЛЕНИЯ, ЛИЦЕНЗИИ, А), ORG), ((ВАС), ORG), ((ДИДЖИТАЛ, БИЗНЕС, ЗВОНЮ, ВАМ, ПО, ПОВОДУ, ПРОДЛЕНИЯ, А, МЫ, СЕЛИ, ОБРАТИЛА, ВНИМАНИЕ, ЧТО, У, ВАС, СРОК, ЗАКАНЧИВАЕТСЯ, \n, МЕНЯ, ЗОВУТ, АНГЕЛИНА, КОМПАНИЯ, ДИДЖИТАЛ, БИЗНЕС, ЗВОНЮ, ВАМ, ПО, ПОВОДУ, ПРОДЛЕНИЯ, ЛИЦЕНЗИИ, А), ORG), ((МАСТЕРА, МЫ), ORG), ((ВАМИ), ORG), ((МАКСИМ, ЗОВУТ), ORG), ((КОМПАНИЯ, КИТОБИЗНЕС, УДОБНО, ГОВОРИТЬ), ORG)]"
uppercase,"[(Ангелина, B-PERSON), (Ангелина, B-PERSON), (Ангелина, B-PERSON), (Максим, B-PERSON)]","[(Зовут, B-PER), (Ангелина, I-PER), (Диджитал, B-PER), (Бизнес, I-PER), (Звоним, I-PER), (Вам, I-PER), (По, I-PER), (А, B-PER), (Мы, I-PER), (С, I-PER), (Серым, I-PER), (Вас, B-PER), (Скоро, I-PER), (Заканчивается, B-LOC), (Зовут, B-PER), (Ангелина, I-PER), (Диджитал, B-PER), (Бизнес, I-PER), (Звоню, I-PER), (Вам, I-PER), (По, I-PER), (А, B-PER), (Мы, I-PER), (Сели, I-PER), (Внимание, B-PER), (Что, I-PER), (Вас, B-PER), (Срок, I-PER), (Заканчивается, I-PER), (Зовут, B-PER), (Ангелина, I-PER), (Диджитал, B-PER), (Бизнес, I-PER), (Звоню, I-PER), (Вам, I-PER), (По, I-PER), (А, B-PER), (Мастера, I-PER), (Мы, I-PER), (С, I-PER), (Вами, I-PER), (Видео, B-PER), (Там, I-PER), (Максим, B-PER), (Зовут, I-PER), (Компания, I-PER), (Китобизнес, I-PER), (Удобно, I-PER), (Говорить, I-PER)]","[((Вам, По, Поводу, Продления, Лицензии, А, Мы, С, Серым, У, Вас, Скоро, Срок, Заканчивается, \n, Меня, Зовут, Ангелина), PER), ((Вам, По, Поводу, Продления, А, Мы, Сели, Обратила, Внимание, Что, У, Вас, Срок, Заканчивается, \n, Меня, Зовут, Ангелина), PER), ((Вам, По, Поводу, Продления, Лицензии, А, Мастера, Мы, С, Вами, Сотрудничали, По, Видео, Там, \n, Добрый, Меня, Максим, Зовут), PER), ((Китобизнес, Удобно, Говорить), PER)]"


Видно, что обнаружить имена смогли модель pavlov_multiligual на датасете, где каждое слово начинается с заглавной буквы, и модель spacy_rus - на датасете, где все слова начинаются с маленькой буквы. Для извлечения имен возьмем модель spacy_rus.

При этом, ни одна из моделей не справилась с извлечением организаций. Однако, поскольку диалоги из датасета проводятся по скрипту, можно извлечь названия компаний с помощью правил. Из тестовых сообщений видно, что название организации идет после слова "компания" - воспользуемся им и его синонимами.

In [226]:
organizations = '(?:компани|организаци|фирм|холдинг|предприяти)\w{0,1}\s*(.*?)(?=\s*(?:звон|удобн|вам|я))'

Для приветствия и прощания тоже удобно использовать правила, т.к. форм приветствия и прощания в официальной речи в русском языке немного.

In [237]:
greetings = 'здравствуйте|добр(?:ый|ое)\s*(?:утро|день|вечер)'
farewells = 'всего\s*доброго|до\s*(?:свидания|скорого|завтра|встреч|понедельник|вторник|сред|четверг|пятница|суббота|воскресень)'

Теперь извлечем данные. Учтем, что менеджер скорее называет компанию в той же фразе, в которой представляется. Кроме того, логично представляться и называть компанию в одной из первых фраз диалога.

In [316]:
greet_ids, bye_ids, name_ids, org_ids = [], [], [], []
names, orgs = [], []

results = []

for id in dialogs_id:
    dlg = scripts[(scripts.dlg_id == id) & (scripts.role == 'manager')]

    greet, bye = dlg[dlg.text.str.contains(greetings)], dlg[dlg.text.str.contains(farewells)]

    if len(greet) > 0:
      greet_ids.extend(greet.index)
      greet = True

    else: greet = False

    if len(bye) > 0:
      bye_ids.extend(bye.index)
      bye = True

    else: bye = False
    
    greet = len(dlg[dlg.text.str.contains(greetings)]) > 0
    bye = len(dlg[dlg.text.str.contains(farewells)]) > 0
    name, org = None, None
    dlg_names = dlg[(dlg.text.str.contains('меня')) & (dlg.text.str.contains('зовут'))]
    dlg_names = dlg_names[dlg_names.line_n < dlg_names.line_n.min() + 2] #имя должно быть в одной из первых реплик менеджера
    name = ner_model_spacy('\n'.join(dlg_names.text.values)).ents
    if name:
      name_ids.extend(dlg_names.index)
      name = str(name[0])
      names.append(name)
      org = dlg_names.text.str.findall(organizations).values[0][0]
      if org: 
        org_ids.extend(dlg_names.index)
        orgs.append(org)

    results.append([greet, bye, name, org])

In [317]:
summary = pd.DataFrame(results, columns = ['greet', 'farewell', 'name', 'org'])
summary.to_csv('dialogs_summary.csv')
summary

,greet,farewell,name,org
0,True,True,ангелина,диджитал бизнес
1,True,True,ангелина,диджитал бизнес
2,True,False,ангелина,диджитал бизнес
3,True,True,максим,китобизнес
4,False,True,(),None
5,False,True,(),None


In [320]:
scripts['insight'] = scripts.index
scripts['insight'] = scripts.insight.apply(lambda s:
    f'greeting={s in greet_ids}, farewell={s in bye_ids}, name={names[name_ids.index(s)] if s in name_ids else None}, \
org={orgs[org_ids.index(s)] if s in org_ids else None}')
scripts.head()

,dlg_id,line_n,role,text,insight
0,0,0,client,алло,"greeting=False, farewell=False, name=None, org=None"
1,0,1,manager,алло здравствуйте,"greeting=True, farewell=False, name=None, org=None"
2,0,2,client,добрый день,"greeting=False, farewell=False, name=None, org=None"
3,0,3,manager,меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,"greeting=False, farewell=False, name=ангелина, org=диджитал бизнес"
4,0,4,client,ага,"greeting=False, farewell=False, name=None, org=None"


In [304]:
#запись инсайтов в csv-файл
scripts.to_csv('test_data.csv')